Initialization and setup stuff:

In [86]:
import psycopg2 as pg
import numpy as np
import scipy
from time import time
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
%matplotlib inline

In [61]:
conn = pg.connect(database="weather", user="weather_ro", host="localhost")
conn.autocommit = True
cur = conn.cursor()

Datetime types in our database are timezone-naive.  The timezone setting decides how naive timestamps should be interpretted.  Ours are UTC rather than local

In [62]:
cur.execute("SET SESSION TIME ZONE UTC;")

Check everything looks good:

In [63]:
cur.execute("SELECT count(*) FROM tblWeatherHistoric;")
print("{} rows found.".format(cur.fetchall()[0][0]))

346108 rows found.


Fetch everything a la historic:

In [125]:
cur.execute("SELECT timestamp, avtemp, instsunhours, instrainfall FROM tblWeatherHistoric ORDER BY timestamp ASC;")
print("Got {} rows.".format(cur.rowcount))
timestamps, av_temps, sunhourss, rainfalls = zip(*cur.fetchall())
t = np.array(timestamps)

Got 346111 rows.


#Mask and process data

In [111]:
av_temps = [np.nan if x is None else np.float(x) for x in av_temps]
av_temps = np.array(av_temps) / 10.0

sunhourss = np.array(sunhourss)/100.0
print("{} invalid sun-entries".format(sum(sunhourss>0.6)))
sunhourss[sunhourss>0.6] = np.nan

rainfalls = np.array(rainfalls) / 1000.0

193 invalid sun-entries


#More Crunching

Work out sunshine per day.  This takes a few minutes.

In [ ]:
day0 = datetime(year=1995,month=6,day=30,hour=0,minute=0,second=0)
days = []
daysunhourss = []
day = day0
while day < datetime(year=2015,month=6,day=30,hour=0,minute=0,second=0):
    days.append(day)
    tomorrow = day + timedelta(days=1)
    timestoday = np.logical_and(t > day, t < tomorrow)
    suntoday = np.nansum(sunhourss[timestoday])
    daysunhourss.append(suntoday)
    day = tomorrow
days = np.array(days)
daysunhourss = np.array(daysunhourss)

Work out max temp per day.  This takes a few minutes

In [120]:
max_daily_temps = np.zeros((days.size))
for idx, day in enumerate(days):
    tomorrow = day + timedelta(days=1)
    timestoday = np.logical_and(t > day, t < tomorrow)
    try: today_max = np.nanmax(av_temps[timestoday])
    except ValueError: today_max = 0.0
    max_daily_temps[idx] = today_max

Cumulative rain per day in mm.  Again, be patient.

In [130]:
daily_rainfall = np.zeros((days.size))
for idx, day in enumerate(days):
    tomorrow = day + timedelta(days=1)
    timestoday = np.logical_and(t > day, t < tomorrow)
    rain_today = np.nansum(rainfalls[timestoday]) / 2.0  # summing mm/hr for half-hourly blocks
    daily_rainfall[idx] = rain_today

Characterise summers

In [148]:
summer = datetime(year=1995, month=6, day=1, hour=0, minute=0, second=0)
summers = []
warm_sunny_days_in_summer = []
hot_sunny_days_in_summer = []
rainy_days_in_summer = []
while summer < datetime(2015, month=1, day=1, hour=0, minute=0, second=0):
    end_summer = summer.replace(month=9)
    days_in_summer = np.logical_and(days > summer, days < end_summer)
    warm = np.nan_to_num(max_daily_temps[days_in_summer]) > 20.0
    hot = np.nan_to_num(max_daily_temps[days_in_summer]) > 25.0
    rainy = np.nan_to_num(daily_rainfall[days_in_summer]) > 1.0
    sunny = np.nansum(np.nan_to_num(daysunhourss) > 7)
    warm_and_sunny = np.logical_and(warm, sunny)
    hot_and_sunny = np.logical_and(hot, sunny)
    
    summers.append(summer)
    warm_sunny_days_in_summer.append(np.nansum(warm_and_sunny))
    hot_sunny_days_in_summer.append(np.nansum(hot_and_sunny))
    rainy_days_in_summer.append(np.nansum(rainy))
    
    summer = summer.replace(year=summer.year+1)

Rate My Summer

In [163]:
for summer, warmsunny, hotsunny, rainy in zip(summers, warm_sunny_days_in_summer,
                                              hot_sunny_days_in_summer,
                                              rainy_days_in_summer):
    score = 3*hotsunny + warmsunny - 4*rainy
    print("{}: Hot&sunny:{: 3}  Warm&sunny:{: 3}  Rainy:{: 3}  Score: {: 4d}".format(summer.year, warmsunny, hotsunny, rainy, score))

1995: Hot&sunny: 57  Warm&sunny: 40  Rainy:  5  Score:  157
1996: Hot&sunny: 70  Warm&sunny: 33  Rainy: 16  Score:  105
1997: Hot&sunny: 69  Warm&sunny: 36  Rainy: 23  Score:   85
1998: Hot&sunny: 60  Warm&sunny: 16  Rainy: 18  Score:   36
1999: Hot&sunny: 66  Warm&sunny: 35  Rainy: 18  Score:   99
2000: Hot&sunny: 63  Warm&sunny: 23  Rainy:  8  Score:  100
2001: Hot&sunny: 68  Warm&sunny: 29  Rainy: 16  Score:   91
2002: Hot&sunny: 67  Warm&sunny: 25  Rainy: 16  Score:   78
2003: Hot&sunny: 82  Warm&sunny: 49  Rainy: 12  Score:  181
2004: Hot&sunny: 82  Warm&sunny: 35  Rainy: 13  Score:  135
2005: Hot&sunny: 75  Warm&sunny: 34  Rainy: 15  Score:  117
2006: Hot&sunny: 81  Warm&sunny: 40  Rainy: 16  Score:  137
2007: Hot&sunny: 65  Warm&sunny: 13  Rainy: 23  Score:   12
2008: Hot&sunny: 70  Warm&sunny: 14  Rainy: 14  Score:   56
2009: Hot&sunny: 34  Warm&sunny:  5  Rainy:  9  Score:   13
2010: Hot&sunny: 66  Warm&sunny: 19  Rainy: 17  Score:   55
2011: Hot&sunny: 51  Warm&sunny:  7  Rai

#Interesting Results

Average temperature over all time

In [ ]:
print("Average temperature: {}".format(np.nanmean(av_temps)))

In [91]:
print("The sunniest day was {} with {} hours".format(
    days[np.nanargmax(daysunhourss)], np.nanmax(daysunhourss)))

The sunniest day was 2005-06-27 00:00:00 with 16.6 hours


In [101]:
sun_seven = np.nansum(np.nan_to_num(daysunhourss) > 7)
sun_ten = np.nansum(np.nan_to_num(daysunhourss) > 10)
print("{} days had more than 7 hours sun, {} more than 10 hours.".format(sun_seven,sun_ten))

1429 days had more than 7 hours sun, 584 more than 10 hours.


In [121]:
warm_days = np.nan_to_num(max_daily_temps) > 20.0
hot_days = np.nan_to_num(max_daily_temps) > 25.0
print("I count {} warm days and {} hot days".format(
    sum(warm_days), sum(hot_days)))

I count 1912 warm days and 629 hot days


In [123]:
warm_and_sunny = np.logical_and(warm_days, sun_seven)
print("I count {} warm and sunny days!".format(sum(warm_and_sunny)))

I count 1912 warm and sunny days!


In [124]:
hot_and_sunny = np.logical_and(hot_days, sun_seven)
print("I count {} hot and sunny days!".format(sum(hot_and_sunny)))

I count 629 hot and sunny days!


In [141]:
rainy = np.nan_to_num(daily_rainfall) > 1.0
print("I count {} rainy days (5mm)".format(sum(rainy)))

I count 1290 rainy days (5mm)


In [166]:
tot = np.nansum(rainfalls)/2000.0
tot2 = np.nansum(daily_rainfall)/1000.0
print("Total rainfall: {}m or {}m".format(tot, tot2))

Total rainfall: 2.4405m or 5.452689m


In [169]:
avesun = np.nanmean(daysunhourss)
print("Average daily sun: {} hours".format(avesun))
coffee_per_day = 100 * 2 # 100 phd students, 2 coffees per day
energy_per_day = coffee_per_day * 150000.0  # ~150kJ to boil a cup of water
energy_per_sunhour = energy_per_day / avesun
# 150 watts per sq meter of panel (inc efficiency) = 540,000 J / hr / m^2
energy_per_sunhour_per_sqmetre = 540000.0 
area_req = energy_per_sunhour / energy_per_sunhour_per_sqmetre
print("Roof PV area required: {}m^2".format(area_req))

Average daily sun: 3.784192607 hours
Roof PV area required: 14.6809534622m^2
